# 神经网络

## 定义网络

In [5]:
import torch
import torch.nn as nn # neural network
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 个输入图像通道，6 个输出通道，5x5 平方卷积
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射函数: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果大小是正方形，可只指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除 batch 维度外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只需定义前向函数，并使用自动微分自动定义反向函数。可以在前向函数中使用任何张量操作。

net.parameters() 返回模型的可学习参数。

In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1 的权重

10
torch.Size([6, 1, 5, 5])


尝试一个随机 32x32 输入：

In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0264,  0.0645,  0.0365,  0.0777, -0.0715,  0.0016, -0.1380, -0.0980,
          0.0867, -0.0040]], grad_fn=<ThAddmmBackward>)


使用随机梯度将所有参数和反向传播的梯度缓冲区归零：

In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

torch.nn 只支持小批量样本。整个torch.nn软件包仅支持小批量样本的输入，而不是单个样本。

例如，nn.Conv2d将采用4维张量的nSamples x nChannels x Height x Width.

如果您有一个样本，只需使用input.unsqueeze(0)添加假批量维度。

- torch.Tensor - 支持自动微分操作例如 backward() 的多维数组。同时含有关于张量的梯度。
- nn.Module - 神经网络模块。
- nn.Parameter - 一种张量，在被指定为 Module 的属性时自动注册为参数。
- autograd.Function - 每个 Tensor 操作都会创建至少一个 Function 节点，该节点连接到创建 Tensor 并对其历史进行编码的函数。

## 损失函数

损失函数将 (output, target) 作为输入，并计算估计值与目标值之间的差距。

nn 包下有几种不同的损失函数。一个简单的损失函数是：nn.MSELoss，它计算输入和目标之间的均方误差（mean-squared error）。

In [9]:
output = net(input)
target = torch.randn(10) # 作为示例
target = target.view(1, -1) # 与输出同样大小
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1802, grad_fn=<MseLossBackward>)


现在，如果您在反向上跟随损失，使用其.grad_fn 属性，您将看到如下所示的计算图：

因此，当调用 loss.backward() 时将求整张图关于损失的微分，所有图中 requires_grad = True 的张量将累积其.grad 张量。

为了说明这一点，我们向后退几步：

In [10]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # 线性
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

## 反向传播

要反向传播误差，需要使用 lost.backward(). 您需要清除现有梯度，否则梯度将积累到现有梯度。

现在我们调用loss.backward()，并查看 conv1 在反向之前和反向之后的偏差梯度。

In [11]:
net.zero_grad() # 清零所有参数的梯度缓冲
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0065, -0.0055, -0.0046,  0.0039,  0.0054, -0.0011])


这就是如何使用损失函数。

## 更新权重

实际运用中最简单的权重更新方法是随机梯度下降 SGD：

weight = weight - learning_rate * gradient

In [13]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

使用神经网络时，可能用到不同的更新规则，例如 SGD、Nseterov-SGD、Adam、RMSProp 等。torch.optim 包中实现了所有这些方法。

In [15]:
import torch.optim as optim
# 构建优化器
optimizer = optim.SGD(net.parameters(), lr = 0.01) # 学习率 lr

# 在训练的循环中：
optimizer.zero_grad() # 清零梯度缓冲
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # 执行更新

梯度缓冲必须使用 optimizer.zero_grad() 手动置为0，因为梯度是累积的。